# Making HTTP Requests

In Python, there are many libraries to make HTTP requests. We will use a 3rd-party library called "requests", which is very easy to use. 

Making a "GET" request is as simple as: 

```python
import requests

res = requests.get(url) # returns a "Response" object
res.content # has the "body" of the response
```

You might need to install the requests library! 

You can do that with the following code in a Jupyter cell: 

```python
! pip install requests
```

Or, if you're using anaconda, optionally you can also do: 

```python
! conda install -c anaconda requests
```

## Parsing JSON data

To parse JSON data in Python, we will use the "json" module: 

```python
import json
```

Read more about the module on the [documentation page](https://docs.python.org/3/library/json.html)!

All we care about for this part is the method "loads", which turns JSON data into a Python object: 

```python
json.loads(my_string_encoded_json)
```

## Pokemon API

There is a simple, open API called "pokeapi" that allows us to make requests and see how to use APIs. Like everything, we first look at the documentation: 

https://pokeapi.co/docs/v2.html

In [3]:
# Let's see how to make a get request to the API: 
import requests
import json
import pandas as pd

res = requests.get('https://pokeapi.co/api/v2/pokemon')
temp = json.loads(res.content)
print(temp)
#names = temp['results']['name']
next_element = temp['next']
print(next_element)
names = temp['results']
names_list = [(i['name'], i['url']) for i in names]
#dictionary = dict(('name', key['name']) for key in names)
#url_list = [i['url'] for i in names]
#dictionary = dict(zip(keys, values))

#print(names_list)
#print(dictionary)

{'count': 964, 'next': 'https://pokeapi.co/api/v2/pokemon?offset=20&limit=20', 'previous': None, 'results': [{'name': 'bulbasaur', 'url': 'https://pokeapi.co/api/v2/pokemon/1/'}, {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'}, {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon/3/'}, {'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon/4/'}, {'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'}, {'name': 'charizard', 'url': 'https://pokeapi.co/api/v2/pokemon/6/'}, {'name': 'squirtle', 'url': 'https://pokeapi.co/api/v2/pokemon/7/'}, {'name': 'wartortle', 'url': 'https://pokeapi.co/api/v2/pokemon/8/'}, {'name': 'blastoise', 'url': 'https://pokeapi.co/api/v2/pokemon/9/'}, {'name': 'caterpie', 'url': 'https://pokeapi.co/api/v2/pokemon/10/'}, {'name': 'metapod', 'url': 'https://pokeapi.co/api/v2/pokemon/11/'}, {'name': 'butterfree', 'url': 'https://pokeapi.co/api/v2/pokemon/12/'}, {'name': 'weedle', 'url': 'https://pokeapi.co/api/v2

In [4]:
# Challenge: 
# Create a Dataframe with all the Pokemon names and their URLs. 

def get_pokes(url):
    # Make the HTTP request to the given url. 
    # Parse the response as json
    # return the "next" and the "results" (as a 2-tuple!)
    # make sure to return a "falsey" value (such as None)
    # if there is not a "next!"
    res = requests.get(url)
    temp = json.loads(res.content)
    names = temp['results']
    names_list = [(i['name'], i['url']) for i in names]
    next_element = temp['next']
    return (next_element, names_list)

def catch_em_all(url):
    pokes = []
    
    # While loop! Like a for-loop, 
    # but goes on for an indetermined amount
    # of time:
    while url:
        url, results = get_pokes(url)
        pokes += results
    return pokes
        
    
list_of_pokes = catch_em_all('https://pokeapi.co/api/v2/pokemon')

# This data is most naturally represented as a list of dictionaries. 
# How can we create a dataframe from a list of dictionaries? 
# Try to find out on your own, from the internet!

# TODO: turn list_of_pokes into a dataframe.
df = pd.DataFrame(list_of_pokes, columns=['name', 'URL'])
print(df)

                        name                                       URL
0                  bulbasaur      https://pokeapi.co/api/v2/pokemon/1/
1                    ivysaur      https://pokeapi.co/api/v2/pokemon/2/
2                   venusaur      https://pokeapi.co/api/v2/pokemon/3/
3                 charmander      https://pokeapi.co/api/v2/pokemon/4/
4                 charmeleon      https://pokeapi.co/api/v2/pokemon/5/
5                  charizard      https://pokeapi.co/api/v2/pokemon/6/
6                   squirtle      https://pokeapi.co/api/v2/pokemon/7/
7                  wartortle      https://pokeapi.co/api/v2/pokemon/8/
8                  blastoise      https://pokeapi.co/api/v2/pokemon/9/
9                   caterpie     https://pokeapi.co/api/v2/pokemon/10/
10                   metapod     https://pokeapi.co/api/v2/pokemon/11/
11                butterfree     https://pokeapi.co/api/v2/pokemon/12/
12                    weedle     https://pokeapi.co/api/v2/pokemon/13/
13    

## Project: Live Exchange Rates

Imagine that you work with financial assets which are denominated in different currencies. You analyze this data regularly, and want to create a "transformation" function that transforms all your assets into EUR prices, based on today's exchange rate. 

Your data with the local-currency-denominated value of each asset lives in a file called "assets.csv" which should be located in the same folder as this notebook. 

Write a "data loading" function that: 

1. Reads the data, given the path to the file. 
2. Returns a dataframe with an additional column that has the assets value in euros, as of today.

Use this free API to get today's exchange rates: https://exchangeratesapi.io/. You will need to read the documentation and try it out to see how it works. 

HINT: Write a separate function to get the current exchange rates! That can be reused!

In [8]:
def get_current_exchange_rate(currency):
    """
    Get today's exchange rate from the API for a given currency
    """
    curr_string = 'https://api.exchangeratesapi.io/latest?base=' + currency
    res = requests.get(curr_string)
    temp = json.loads(res.content)
    rates = temp['rates']
    return rates

# Test
#get_current_exchange_rate("USD")

def converter(path,currency):
    """
    Convert the local-currency-denominated value of assets
    located in a file in a given path,
    to a given currency
    and add a new column containing its converted value
    """
    # Get exchange rates and convert into dataframe
    rates = get_current_exchange_rate(currency)
    df = pd.DataFrame.from_dict(rates, orient='index').reset_index()
    df.columns = ['currency', 'ex_rate']
    
    # Read in the asset file
    assets = pd.read_csv(path+'./assets.csv')
    
    # Merge files and create new variable
    full = assets.merge(df, left_on='curr', right_on='currency',how = 'left')
    varname = currency + '_value'
    full[varname] = full['value']/full['ex_rate']
    full = full.drop(columns=['currency','ex_rate'])
    
    return full

# Test functions
testdata = converter(path='',currency='EUR')
print(testdata.head())


       value curr  EUR_value
0  48.910052  THB   1.456523
1  16.505115  THB   0.491516
2  30.370579  INR   0.385558
3  14.126967  SEK   1.307931
4  23.406904  HKD   2.687977
